In [25]:
import pandas as pd
import datetime
import os
import pytz
utc = pytz.utc
AZtz = pytz.timezone("US/Arizona")

dataFileColumns = ["timestamp", "stage", "endTimestamp"]
outputFileName = "appleWatch6pillowSleepStages.parquet.gzip"

#get list of files in current directroy
files = [f for f in os.listdir("./") if os.path.isfile(f)]

#if the output file isn't already made, make it
if not (outputFileName in files):
    print("making new df")
    emptydf = pd.DataFrame(columns = dataFileColumns)
    emptydf.set_index('timestamp')
    print(emptydf)
        
    #save as a parquet file
    emptydf.to_parquet(outputFileName, compression='gzip') 


df = pd.read_parquet(outputFileName)

print(df)

making new df
Empty DataFrame
Columns: [timestamp, stage, endTimestamp]
Index: []
Empty DataFrame
Columns: [timestamp, stage, endTimestamp]
Index: []


In [26]:
datDir = "./appleWatch/PillowData.txt"

with open (datDir, "r") as myfile:
    dataSTRs=myfile.readlines()

#print (dataSTRs[24])

for i in range(len(dataSTRs)):
    #print(dataSTRs[i])
    if dataSTRs[i].strip() == "ZSLEEPSTAGEDATAPOINT":
        exsleepSTRs = dataSTRs[i+1:]

for i in range(len(exsleepSTRs)):
    if exsleepSTRs[i].strip() == "ZSNOOZELAB":
        sleepSTRs = exsleepSTRs[:i-1]

    

In [27]:
tempdf =pd.DataFrame(columns = ["timestamp", "value"])
unix = datetime.datetime(1970, 1, 1)  # UTC
cocoa = datetime.datetime(2001, 1, 1)  # UTC

delta = cocoa - unix  # timedelta instance

for s in sleepSTRs:
    l = s.split("Z")
    #print(l)
    time = AZtz.localize(datetime.datetime.fromtimestamp(int(l[7].split(".")[0][12:])) + delta).astimezone(utc)
    #print(time)
    tempdf.loc[time] = [time,int(l[5][-3])]

tempdf.sort_index(inplace=True)


In [28]:
#print(tempdf.head(20))

In [29]:
stageStartTime = tempdf.iloc[0]['timestamp']
toadd = []
toadd.append(stageStartTime)
toadd.append(int(tempdf.iloc[0]['value']))

for i in range(1,len(tempdf)):
    if tempdf.iloc[i]['value'] != tempdf.iloc[i-1]['value']:
        currentStageTime = tempdf.iloc[i]['timestamp']
        toadd.append(currentStageTime)
        df.loc[stageStartTime] = toadd
        toadd = []
        stageStartTime = currentStageTime
        toadd.append(stageStartTime)
        toadd.append(int(tempdf.iloc[i]['value']))





In [30]:
print(df)
df.to_parquet(outputFileName, compression='gzip') 

                                          timestamp stage  \
index                                                       
2021-07-20 08:34:22+00:00 2021-07-20 08:34:22+00:00     1   
2021-07-20 08:45:52+00:00 2021-07-20 08:45:52+00:00     0   
2021-07-20 09:31:34+00:00 2021-07-20 09:31:34+00:00     1   
2021-07-20 09:49:40+00:00 2021-07-20 09:49:40+00:00     3   
2021-07-20 10:02:45+00:00 2021-07-20 10:02:45+00:00     1   
...                                             ...   ...   
2021-08-09 13:35:31+00:00 2021-08-09 13:35:31+00:00     0   
2021-08-09 13:49:16+00:00 2021-08-09 13:49:16+00:00     2   
2021-08-09 14:34:44+00:00 2021-08-09 14:34:44+00:00     0   
2021-08-09 14:47:53+00:00 2021-08-09 14:47:53+00:00     1   
2021-08-09 15:17:44+00:00 2021-08-09 15:17:44+00:00     0   

                                       endTimestamp  
index                                                
2021-07-20 08:34:22+00:00 2021-07-20 08:45:52+00:00  
2021-07-20 08:45:52+00:00 2021-07-20 09:31:3

In [31]:
#import it as one big string
#split at ZSLEEPSTAGEDATAPOINT 
#then split at ZSNOOZELAB



#each line will be it's own stage split at Z

#i'm going to add each line to a temporary DF 
#sort by timestamp and reindex
#iterate over indexes calculate the time in the sleep stage coalesing neighboring sleep stages
#save new rows to the proper df to be saved